# Hybrid search for the metadata catalogue of the Canton of Zurich

In this notebook we prepare the data for our search application and setup the search index.

- We load the data from the API.
- We prepare the data and lemmatize the text data (for lexical search).
- We embed via the OpenAI embedding API.
- We test the embeddings with cosine similarity.
- We setup the Weaviate index.
- We create a collection with our data.
- We test the index in regard to lexical, and vector search as well as the combination of both - hybrid search. The latter is what we use in the app.


# Imports


In [1]:
import pandas as pd
from pandarallel import pandarallel
import numpy as np

pd.options.mode.chained_assignment = None
pd.options.display.max_rows = 500
pd.options.display.max_seq_items = 500
pandarallel.initialize(progress_bar=False)

from time import time
import time
import os
import re
import requests
from dotenv import load_dotenv

import pyarrow.parquet as pq

import warnings
from bs4 import MarkupResemblesLocatorWarning
from sklearn.metrics.pairwise import cosine_similarity
import weaviate
from weaviate.classes.config import Property, DataType
import weaviate.classes as wvc
import weaviate.classes.config as wc

import spacy
from openai import OpenAI
import json

warnings.filterwarnings("ignore")
warnings.filterwarnings("ignore", category=MarkupResemblesLocatorWarning)

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
client_openai = OpenAI(api_key=OPENAI_API_KEY)

INFO: Pandarallel will run on 2 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


**Constants**


In [3]:
DATA_FOLDER = "_data/"
DATASETS = DATA_FOLDER + "01_data.parq"
DATA_WITH_EMBEDDINGS = DATA_FOLDER + "02_data_embedded.parq"

BASELINK_API = "https://daten.berlin.de/datensaetze/"

# Dataset links are composed of this baselink and the identifier for each dataset.
BASELINK_DATASHOP = (
    "https://datenregister.berlin.de/api/3/action/current_package_list_with_resources"
)

In [61]:
# FUNCTIONS ------------------------------------------------------------------ #


# Retrieving data from the Berlin data register API, processing it, and saving it to CSV files
def get_full_package_list(limit=500, sleep=2):
    """Get full package list from CKAN API"""
    offset = 0
    frames = []
    while True:
        print(f"{offset} packages retrieved.")
        url = BASELINK_DATASHOP + f"?limit={limit}&offset={offset}"
        res = requests.get(url)
        data = json.loads(res.content)
        if data["result"] == []:
            break
        data = pd.DataFrame(pd.json_normalize(data["result"]))
        frames.append(data)
        offset += limit
        time.sleep(sleep)
    data = pd.concat(frames)
    data.reset_index(drop=True, inplace=True)
    return data


all_packages = get_full_package_list()

0 packages retrieved.
500 packages retrieved.
1000 packages retrieved.
1500 packages retrieved.
2000 packages retrieved.
2500 packages retrieved.
3000 packages retrieved.
3500 packages retrieved.


In [62]:
raw = all_packages.copy()

In [63]:
# Convert the DataFrame to a list of dictionaries
raw = (all_packages.copy()).to_dict(orient="records")

# Convert the DataFrame to a list of strings
raw = (
    (all_packages.copy()).apply(lambda row: " ".join(row.astype(str)), axis=1).tolist()
)

In [64]:
# Print the result
print(raw[0])

print(type(raw[0]))

Senatsverwaltung für Arbeit, Soziales, Gleichstellung, Intergartion, Vielfalt und Antidiskriminierung bildungszeit@senasgiva.berlin.de simplesearch datensatz 50e8a594-1945-487e-b77e-3baa14f6a3c7 2015-10-19 2024-02-14 Berlin Berlin a6828125-ae95-404f-9fb0-52eb91302fd8 True cc-zero Creative Commons Zero https://opendefinition.org/licenses/cc-zero/ n.n. bildungszeit@senasgiva.berlin.de 2023-11-01T14:01:17.996380 2024-08-17T00:28:35.151018 simple_search_wwwberlindesenarbeitweiterbildungbildungszeitsuche In Berlin anerkannte Weiterbildungsveranstaltungen. 5 4 e2f57191-ec25-4249-ab08-98366b339ef5 False active 2024-01-01 2024-01-31 Keine Suche nach anerkannten Veranstaltungen dataset https://www.berlin.de/sen/arbeit/weiterbildung/bildungszeit/suche/ None [{'description': '', 'display_name': 'Arbeitsmarkt', 'id': 'e727a5a0-adae-465f-8c5b-1d84b681cc0d', 'image_display_url': '', 'name': 'arbeit', 'title': 'Arbeitsmarkt'}] [{'cache_last_updated': None, 'cache_url': None, 'created': '2024-08-17T00

In [65]:
# def clean_text(d):
#     """Clean text. Remove square brackets that often enclose units or other information. Replace multiple spaces with single spaces and strip leading and trailing spaces.

#     Args:
#         d (str): Text to clean.

#     Returns:
#         str: Cleaned text.
#     """
#     # Remove square brackets.
#     d = re.sub(r"(\[|\])", " ", d)
#     # Replace multiple spaces with single space.
#     d = re.sub(r"[\s]+", " ", d)
#     d = d.strip()
#     return d

In [66]:
type(all_packages)

pandas.core.frame.DataFrame

In [67]:
all_packages.columns

Index(['author', 'author_email', 'berlin_source', 'berlin_type',
       'creator_user_id', 'date_released', 'date_updated',
       'geographical_coverage', 'geographical_granularity', 'id', 'isopen',
       'license_id', 'license_title', 'license_url', 'maintainer',
       'maintainer_email', 'metadata_created', 'metadata_modified', 'name',
       'notes', 'num_resources', 'num_tags', 'owner_org', 'private', 'state',
       'temporal_coverage_from', 'temporal_coverage_to',
       'temporal_granularity', 'title', 'type', 'url', 'version', 'groups',
       'resources', 'tags', 'relationships_as_subject',
       'relationships_as_object', 'organization.id', 'organization.name',
       'organization.title', 'organization.type', 'organization.description',
       'organization.image_url', 'organization.created',
       'organization.is_organization', 'organization.approval_status',
       'organization.state', 'attribution_text', 'hvd_category',
       'preview_image', 'sample_record', 'use

In [68]:
all_packages.head()

,author,author_email,berlin_source,berlin_type,creator_user_id,date_released,date_updated,geographical_coverage,geographical_granularity,id,...,organization.created,organization.is_organization,organization.approval_status,organization.state,attribution_text,hvd_category,preview_image,sample_record,username,extras
0,"Senatsverwaltung für Arbeit, Soziales, Gleichs...",bildungszeit@senasgiva.berlin.de,simplesearch,datensatz,50e8a594-1945-487e-b77e-3baa14f6a3c7,2015-10-19,2024-02-14,Berlin,Berlin,a6828125-ae95-404f-9fb0-52eb91302fd8,...,2016-01-29T17:11:35.360140,True,approved,active,NaN,NaN,NaN,NaN,NaN,NaN
1,VBB - Verkehrsverbund Berlin-Brandenburg GmbH,None,webform,datensatz,b136bb34-d39b-4be2-8dd7-8779c2d92fe6,2013-07-01,2024-08-16,Berlin,GPS-Koordinaten,96643363-971f-43f9-b93e-e721c5b84872,...,2016-01-29T17:11:40.469439,True,approved,active,VBB Verkehrsverbund Berlin-Brandenburg GmbH,,,,NaN,NaN
2,LAGeSo,webmaster@lageso.berlin.de,simplesearch,datensatz,b1e7b5de-7dab-497a-ba76-867bf93f22b3,2015-07-22,2024-07-26,Berlin,Berlin,4bd75f18-20bf-45e9-af17-3bf03ab6dc58,...,2016-01-29T17:11:35.360140,True,approved,active,NaN,NaN,NaN,NaN,NaN,NaN
3,Servicezentrum der Berliner Volkshochschulen,None,webform,datensatz,49414a9f-43ba-4e0e-b6ec-f0cfb7d791f4,2019-10-14,2024-08-14,Berlin,Berlin,f90b3f13-b3bf-4420-88d0-c6397efc62b9,...,2018-11-14T13:58:47.621467,True,approved,active,,,,,NaN,NaN
4,"Senatsverwaltung für Wirtschaft, Energie und B...",Abdulkadir.Dagci@senweb.berlin.de,simplesearch,datensatz,b1e7b5de-7dab-497a-ba76-867bf93f22b3,2014-04-15,2024-08-14,Berlin,Berlin,6a2acf9e-e911-4d78-b571-c75091d07086,...,2016-01-29T17:11:35.360140,True,approved,active,NaN,NaN,NaN,NaN,NaN,NaN


# Load and parse data


In [69]:
def parse_and_prepare_metadata(df):
    """Parse and prepare metadata for all available datasets.

    Args:
        df (pd.DataFrame): Raw metadata for all available datasets.

    Returns:
        pd.DataFrame: Parsed and prepared metadata for all datasets.
    """

    # This function processes the 'resources' column to extract distribution info and check for PDFs
    def process_distribution(resources):
        tmp_res = []
        is_pdf = False
        if isinstance(resources, list):
            for resource in resources:
                if isinstance(resource, dict):
                    if resource.get("format") == "PDF":
                        is_pdf = True
                    # Convert to string and use empty string if None
                    name = str(resource.get("name", "")) or ""
                    description = str(resource.get("description", "")) or ""
                    tmp_res.append(name)
                    tmp_res.append(description)
        # Filter out empty strings before joining
        return " ".join(filter(bool, tmp_res)), is_pdf

    # Create a new DataFrame with the required columns
    new_df = pd.DataFrame()

    # CHANGES AND EXPLANATIONS:

    # 1. Using 'id' instead of 'identifier'
    # The original function expected 'identifier', but the DataFrame has 'id'
    new_df["identifier"] = df["id"]

    # 2. Constructing the link using 'id'
    # This remains the same, but now uses 'id' instead of 'identifier'
    new_df["link"] = BASELINK_DATASHOP + df["id"]

    # 3. Title remains the same, just stripped of whitespace
    new_df["title"] = df["title"].str.strip()

    # 4. Using 'notes' for description instead of 'description'
    # The DataFrame has 'notes' which likely contains the description
    new_df["description"] = (
        df["notes"]
        .fillna("")
        .apply(lambda x: x.strip() if x not in ["NA", "keine", "null"] else "")
    )

    # 5. Processing 'tags' for keywords
    # The original function expected 'keyword', but the DataFrame has 'tags'
    # Assuming 'tags' is a list of dictionaries with a 'name' key
    new_df["keyword"] = df["tags"].apply(
        lambda x: (
            " ".join([str(tag.get("name", "")) for tag in x])
            if isinstance(x, list)
            else ""
        )
    )

    # 6. Processing 'resources' for distribution and is_study
    # The original function expected a 'distribution' column, which doesn't exist in the DataFrame
    # Instead, we process the 'resources' column to extract this information
    distribution_and_study = df["resources"].apply(process_distribution)
    new_df["distribution"] = distribution_and_study.apply(lambda x: x[0])
    new_df["is_study"] = distribution_and_study.apply(lambda x: x[1])

    return new_df


# OVERALL CHANGES:
# - The function now takes a DataFrame as input instead of a list of dictionaries
# - It maps the existing columns in the DataFrame to the required output format
# - It handles missing or differently named columns (like 'notes' for description)
# - It processes complex columns like 'tags' and 'resources' to extract required information
# - The former 'theme' column is is deleted as there's no corresponding data in the input DataFrame
# - In the `process_distribution` function:
#   * We now convert 'name' and 'description' to strings and use an empty string if they are None
#   * We use `filter(bool, tmp_res)` before joining to remove any empty strings
# - In the 'keyword' processing:
#   * We've added a `str()` conversion and `.get()` method to handle potential None values in the tag names

# These changes are necessary because the structure of the input data (your DataFrame)
# doesn't exactly match what the original function expected. This new version bridges
# that gap, allowing you to process your DataFrame and get the output in the format
# you originally designed, despite the differences in input structure.
# The additional error handling for None values and empty strings ensures that the function
# can process data with missing or null values without raising exceptions.

In [70]:
all_packages.head()

,author,author_email,berlin_source,berlin_type,creator_user_id,date_released,date_updated,geographical_coverage,geographical_granularity,id,...,organization.created,organization.is_organization,organization.approval_status,organization.state,attribution_text,hvd_category,preview_image,sample_record,username,extras
0,"Senatsverwaltung für Arbeit, Soziales, Gleichs...",bildungszeit@senasgiva.berlin.de,simplesearch,datensatz,50e8a594-1945-487e-b77e-3baa14f6a3c7,2015-10-19,2024-02-14,Berlin,Berlin,a6828125-ae95-404f-9fb0-52eb91302fd8,...,2016-01-29T17:11:35.360140,True,approved,active,NaN,NaN,NaN,NaN,NaN,NaN
1,VBB - Verkehrsverbund Berlin-Brandenburg GmbH,None,webform,datensatz,b136bb34-d39b-4be2-8dd7-8779c2d92fe6,2013-07-01,2024-08-16,Berlin,GPS-Koordinaten,96643363-971f-43f9-b93e-e721c5b84872,...,2016-01-29T17:11:40.469439,True,approved,active,VBB Verkehrsverbund Berlin-Brandenburg GmbH,,,,NaN,NaN
2,LAGeSo,webmaster@lageso.berlin.de,simplesearch,datensatz,b1e7b5de-7dab-497a-ba76-867bf93f22b3,2015-07-22,2024-07-26,Berlin,Berlin,4bd75f18-20bf-45e9-af17-3bf03ab6dc58,...,2016-01-29T17:11:35.360140,True,approved,active,NaN,NaN,NaN,NaN,NaN,NaN
3,Servicezentrum der Berliner Volkshochschulen,None,webform,datensatz,49414a9f-43ba-4e0e-b6ec-f0cfb7d791f4,2019-10-14,2024-08-14,Berlin,Berlin,f90b3f13-b3bf-4420-88d0-c6397efc62b9,...,2018-11-14T13:58:47.621467,True,approved,active,,,,,NaN,NaN
4,"Senatsverwaltung für Wirtschaft, Energie und B...",Abdulkadir.Dagci@senweb.berlin.de,simplesearch,datensatz,b1e7b5de-7dab-497a-ba76-867bf93f22b3,2014-04-15,2024-08-14,Berlin,Berlin,6a2acf9e-e911-4d78-b571-c75091d07086,...,2016-01-29T17:11:35.360140,True,approved,active,NaN,NaN,NaN,NaN,NaN,NaN


In [71]:
df = all_packages.loc[
    ~all_packages.apply(lambda x: x.astype(str).str.isspace().all(), axis=1)
]

In [72]:
df_parsed = parse_and_prepare_metadata(df)

In [73]:
df_parsed.head()

,identifier,link,title,description,keyword,distribution,is_study
0,a6828125-ae95-404f-9fb0-52eb91302fd8,https://datenregister.berlin.de/api/3/action/c...,Suche nach anerkannten Veranstaltungen,In Berlin anerkannte Weiterbildungsveranstaltu...,bildungszeit simplesearch weiterbildung weiter...,"None Webseite von ""Suche nach anerkannten Vera...",False
1,96643363-971f-43f9-b93e-e721c5b84872,https://datenregister.berlin.de/api/3/action/c...,VBB-Fahrplandaten via GTFS,Der Verkehrsverbund Berlin-Brandenburg (VBB) s...,Bahn Bus Fahrplan Fähre S-Bahn Straßenbahn Tra...,VBB-Fahrplandaten,False
2,4bd75f18-20bf-45e9-af17-3bf03ab6dc58,https://datenregister.berlin.de/api/3/action/c...,Liste der Badestellen,Die auf der Badestellenkarte für Berlin ausgew...,badegewässer baden badeseen simplesearch,"None Webseite von ""Liste der Badestellen"". Hie...",False
3,f90b3f13-b3bf-4420-88d0-c6397efc62b9,https://datenregister.berlin.de/api/3/action/c...,Kurse der Berliner Volkshochschulen,Hier kann man alle aktuellen Kursangebote der ...,Dienstleistungen im Bereich Bildung Volkshochs...,Aktuelle Berliner Volkshochschul Kurse Dieser ...,False
4,6a2acf9e-e911-4d78-b571-c75091d07086,https://datenregister.berlin.de/api/3/action/c...,Berliner und Brandenburger Straßen- und Volksf...,Berliner Straßen- und Volksfeste,simplesearch strassenfeste straßenfeste volksf...,"None Webseite von ""Berliner und Brandenburger ...",False


In [74]:
df = df_parsed.copy()

In [75]:
# raw = retrieve_mdv_metadata()
# df = parse_and_prepare_metadata(raw)
df.title = df.title.apply(lambda x: x.strip())
df.description = df.description.apply(lambda x: x.strip())

# Combine title and description for embedding.
df["text_for_embedding"] = df.title + " " + df.description
df["text_for_embedding"] = df["text_for_embedding"].apply(lambda x: x.strip())
df.to_parquet(DATASETS)
df.info(memory_usage="deep")
df.head(5)

<class 'pandas.core.frame.DataFrame'>
Index: 3311 entries, 0 to 3310
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   identifier          3311 non-null   object
 1   link                3311 non-null   object
 2   title               3311 non-null   object
 3   description         3311 non-null   object
 4   keyword             3311 non-null   object
 5   distribution        3311 non-null   object
 6   is_study            3311 non-null   bool  
 7   text_for_embedding  3311 non-null   object
dtypes: bool(1), object(7)
memory usage: 9.0 MB


,identifier,link,title,description,keyword,distribution,is_study,text_for_embedding
0,a6828125-ae95-404f-9fb0-52eb91302fd8,https://datenregister.berlin.de/api/3/action/c...,Suche nach anerkannten Veranstaltungen,In Berlin anerkannte Weiterbildungsveranstaltu...,bildungszeit simplesearch weiterbildung weiter...,"None Webseite von ""Suche nach anerkannten Vera...",False,Suche nach anerkannten Veranstaltungen In Berl...
1,96643363-971f-43f9-b93e-e721c5b84872,https://datenregister.berlin.de/api/3/action/c...,VBB-Fahrplandaten via GTFS,Der Verkehrsverbund Berlin-Brandenburg (VBB) s...,Bahn Bus Fahrplan Fähre S-Bahn Straßenbahn Tra...,VBB-Fahrplandaten,False,VBB-Fahrplandaten via GTFS Der Verkehrsverbund...
2,4bd75f18-20bf-45e9-af17-3bf03ab6dc58,https://datenregister.berlin.de/api/3/action/c...,Liste der Badestellen,Die auf der Badestellenkarte für Berlin ausgew...,badegewässer baden badeseen simplesearch,"None Webseite von ""Liste der Badestellen"". Hie...",False,Liste der Badestellen Die auf der Badestellenk...
3,f90b3f13-b3bf-4420-88d0-c6397efc62b9,https://datenregister.berlin.de/api/3/action/c...,Kurse der Berliner Volkshochschulen,Hier kann man alle aktuellen Kursangebote der ...,Dienstleistungen im Bereich Bildung Volkshochs...,Aktuelle Berliner Volkshochschul Kurse Dieser ...,False,Kurse der Berliner Volkshochschulen Hier kann ...
4,6a2acf9e-e911-4d78-b571-c75091d07086,https://datenregister.berlin.de/api/3/action/c...,Berliner und Brandenburger Straßen- und Volksf...,Berliner Straßen- und Volksfeste,simplesearch strassenfeste straßenfeste volksf...,"None Webseite von ""Berliner und Brandenburger ...",False,Berliner und Brandenburger Straßen- und Volksf...


# Lemmatize text


In [76]:
# nlp = spacy.load("de_core_news_lg")
nlp = spacy.load("de_core_news_sm", disable=["ner", "parser"])

LETTERS_AND_DIGITS = re.compile(r"[^a-zäüöA-ZÜÄÖ0-9.]")
MULTIPLE_SPACES = re.compile(r"\s+")


def prepare_for_lexical_search(text, lower=False, remove_umlauts=False):
    """Lemmatize text, and optionally lower case and remove umlauts for lexical search.

    Args:
        text (str): Text to process.

    Keyword Arguments:
        lower (bool): Lower case text (default: {True}).
        remove_umlauts (bool): Remove umlauts from text (default: {True}).

    Returns:
        str: Lemmatized text, optionally lower cased, and without umlauts.
    """
    doc = nlp(text)
    text = " ".join([token.lemma_ if token.is_alpha else token.text for token in doc])
    text = re.sub(LETTERS_AND_DIGITS, " ", text)
    text = re.sub(MULTIPLE_SPACES, " ", text)
    if lower:
        text = text.lower()
    if remove_umlauts:
        text = text.replace("ä", "ae").replace("ö", "oe").replace("ü", "ue")
    return text

In [77]:
df["title_lemma"] = df.title.parallel_apply(lambda x: prepare_for_lexical_search(x))
df["description_lemma"] = df.description.parallel_apply(
    lambda x: prepare_for_lexical_search(x)
)

df["distribution_lemma"] = df.distribution.parallel_apply(
    lambda x: prepare_for_lexical_search(x)
)

Have a quick look at the data.


In [78]:
# Using transposed view to give a better overview of the data
df.sample(2).T

,2655,3080
identifier,043a8358-a407-4ba4-bc66-057d71bb4e9a,46e30354-b019-4137-8166-975dbce006b5
link,https://datenregister.berlin.de/api/3/action/c...,https://datenregister.berlin.de/api/3/action/c...
title,Einwohnerinnen und Einwohner in den Ortsteilen...,Gesundheitsberichterstattung Berlin: Kosten ->...
description,Einwohnerinnen und Einwohner in den Ortsteilen...,Bereitstellung von 54 Dokumenten (51 XLS-Datei...
keyword,Einwohner Ortsteile kleinräumige Einwohnerzahl,Behandlungsfälle Berlin Berufsgruppen Beschäft...
distribution,Datenmatrix Datenmatrix Metadaten Metadaten,None Zugriff über das Gesundheits- und Soziali...
is_study,True,False
text_for_embedding,Einwohnerinnen und Einwohner in den Ortsteilen...,Gesundheitsberichterstattung Berlin: Kosten ->...
title_lemma,Einwohnerin und Einwohner in der Ortsteil Berl...,Gesundheitsberichterstattung Berlin Kosten Kos...
description_lemma,Einwohnerin und Einwohner in der Ortsteil Berl...,Bereitstellung von 54 Dokument 51 XLS Dateien ...


# Embed data


For our prototype app we use OpenAIs embeddings for convienience.

We also tested these open source models with [SentenceTransformers](https://sbert.net/) with very good results:

- [PM-AI/bi-encoder_msmarco_bert-base_german](https://huggingface.co/PM-AI/bi-encoder_msmarco_bert-base_german) - 350 tokens context length
- [Jina AI jina-embeddings-v2-base-de](https://huggingface.co/jinaai/jina-embeddings-v2-base-de) - 8192 tokens context length


In [79]:
def embed_with_openai(texts, model="text-embedding-3-small"):
    """Embed text using OpenAIs embedding API.

    Args:
        texts (list): List of texts to embed.
        model (str): OpenAI model to use (default: {"text-embedding-3-small"}).

    Returns:
        list: List of embeddings.
    """
    response = client_openai.embeddings.create(input=texts, model=model)
    return [x.embedding for x in response.data]

In [80]:
df.text_for_embedding.values

array(['Suche nach anerkannten Veranstaltungen In Berlin anerkannte Weiterbildungsveranstaltungen.',
       'VBB-Fahrplandaten via GTFS Der Verkehrsverbund Berlin-Brandenburg (VBB) stellt regelmäßig die aktuellen Bus- und Bahn-Fahrplandaten aus Berlin und Brandenburg im GTFS-Format zur Verfügung. \r\nMehr zu dem Format ist unter https://developers.google.com/transit/gtfs/ (auf Englisch)) zu finden.\r\nDeutschlandweite Haltestellen (DHID) können im zentralen Haltestellenverzeichnis (https://zhv.wvigmbh.de) abgerufen werden.\r\nVBB-Logos für die erforderliche Namensnennung „VBB Verkehrsverbund Berlin-Brandenburg GmbH“ gibt es unter https://www.vbb.de/presse/media-service/logos?slug=logos.\r\nVeraltete Fahrplandaten können unter api@vbb.de angefragt werden.',
       'Liste der Badestellen Die auf der Badestellenkarte für Berlin ausgewiesenen Bäder und Badestellen werden regelmäßig hinsichtlich der Badegewässerqualität vom 15. Mai bis 15. September (Badesaison) eines jeden Jahres in 14tägi

In [82]:
import pandas as pd
import numpy as np

# Assuming your dataframe is named 'df'


def check_column(df, column_name="text_for_embedding"):
    print(f"Checking column: {column_name}")

    # Check data type
    print(f"\nData type: {df[column_name].dtype}")

    # Check for null values
    null_count = df[column_name].isnull().sum()
    print(f"\nNull values: {null_count}")

    # Check for empty strings
    empty_count = (df[column_name] == "").sum()
    print(f"Empty strings: {empty_count}")

    # Check for whitespace-only strings
    whitespace_count = (df[column_name].str.isspace()).sum()
    print(f"Whitespace-only strings: {whitespace_count}")

    # Get unique data types within the column
    unique_types = df[column_name].apply(type).unique()
    print("\nUnique data types in the column:")
    for t in unique_types:
        print(f"- {t}")

    # Sample of values
    print("\nSample values:")
    print(df[column_name].sample(min(5, len(df))).to_string())

    # Statistics
    print("\nValue counts:")
    print(df[column_name].value_counts().head())

    print("\nBasic statistics:")
    print(df[column_name].describe())


# Run the check
check_column(df)

# If you want to check a different column, you can specify it:
# check_column(df, 'some_other_column')

Checking column: text_for_embedding

Data type: object

Null values: 0
Empty strings: 0
Whitespace-only strings: 0

Unique data types in the column:
- <class 'str'>

Sample values:
481     Verwaltungseinheiten im INSPIRE-Datenmodell - ...
2962    Gesundheitsberichterstattung Berlin: Gesundhei...
658     Standardeinspeiseprofil BHKW 2020 Betreiber vo...
1820    Langjährige Entwicklung Luftqualität NOx-Emiss...
1560    Jugendarbeitslosigkeit 2018 - [WMS] Anteil der...

Value counts:
text_for_embedding
Senatsvorlagen der Senatsverwaltung für Wirtschaft, Energie und Betriebe Parlamentsdokumentation                                                                                                                                                                               2
Prüfberichte der Berliner Heimaufsicht Die Heimaufsicht ist verpflichtet, Prüfberichte über die Ergebnisse der von ihr durchgeführten Prüfungen zu erstellen und diese Prüfberichte zu veröffentlichen. Die veröffentlichten P

In [6]:
from openai import OpenAI
import time
from typing import List, Any
import numpy as np

client = OpenAI()  # Ensure your API key is set in environment variables


def embed_with_openai(
    texts: List[str],
    model: str = "text-embedding-3-small",
    batch_size: int = 100,
    max_retries: int = 3,
    retry_delay: float = 1.0,
) -> List[List[float]]:
    """
    Embed text using OpenAI's embedding API with batching, retries, and error handling.

    Args:
        texts (List[str]): List of texts to embed.
        model (str): OpenAI model to use (default: "text-embedding-3-small").
        batch_size (int): Number of texts to process in each API call (default: 100).
        max_retries (int): Maximum number of retries for failed API calls (default: 3).
        retry_delay (float): Delay in seconds between retries (default: 1.0).

    Returns:
        List[List[float]]: List of embeddings.

    Raises:
        ValueError: If input texts are not valid.
        RuntimeError: If all retries fail.
    """
    if not texts or not all(isinstance(text, str) for text in texts):
        raise ValueError("Input must be a non-empty list of strings")

    all_embeddings = []

    for i in range(0, len(texts), batch_size):
        batch = texts[i : i + batch_size]

        for attempt in range(max_retries):
            try:
                response = client.embeddings.create(input=batch, model=model)
                batch_embeddings = [item.embedding for item in response.data]
                all_embeddings.extend(batch_embeddings)
                break
            except Exception as e:
                print(
                    f"Error on attempt {attempt + 1} for batch {i//batch_size + 1}: {str(e)}"
                )
                if attempt == max_retries - 1:
                    raise RuntimeError(
                        f"Failed to embed batch after {max_retries} attempts"
                    ) from e
                time.sleep(retry_delay * (2**attempt))  # Exponential backoff

    # Verify the output
    if len(all_embeddings) != len(texts):
        raise RuntimeError(
            f"Mismatch in number of embeddings ({len(all_embeddings)}) and input texts ({len(texts)})"
        )

    return all_embeddings


# Usage example
def process_dataframe_embeddings(df, text_column="text_for_embedding"):
    try:
        texts = df[text_column].astype(str).tolist()
        embeddings = embed_with_openai(texts)
        df["embedding_openai"] = embeddings
        print(f"Successfully embedded {len(embeddings)} texts")
    except Exception as e:
        print(f"Error in embedding process: {str(e)}")
        # Optionally, save problematic texts for further investigation
        df[df[text_column].apply(lambda x: not isinstance(x, str) or not x)].to_csv(
            "problematic_texts.csv", index=False
        )
    return df


# Example usage
# df = process_dataframe_embeddings(df)

In [89]:
import pandas as pd
import numpy as np


def process_dataframe_embeddings(texts, model="text-embedding-3-small"):
    try:
        # Ensure texts are strings
        texts = [str(text) for text in texts]

        # Get embeddings
        embeddings = embed_with_openai(texts, model)

        # Return the embeddings directly
        return embeddings

    except Exception as e:
        print(f"Error in embedding process: {str(e)}")
        # Save problematic texts
        pd.DataFrame(
            {
                "problematic_text": [
                    text for text in texts if not isinstance(text, str) or not text
                ]
            }
        ).to_csv("problematic_texts.csv", index=False)
        return None


# Usage
embeddings = process_dataframe_embeddings(df.text_for_embedding.values)

if embeddings is not None:
    df["embedding_openai"] = embeddings

In [91]:
df["embedding_openai"] = process_dataframe_embeddings(df)

Successfully embedded 3311 texts


ValueError: Columns must be same length as key

In [92]:
df.columns

Index(['identifier', 'link', 'title', 'description', 'keyword', 'distribution',
       'is_study', 'text_for_embedding', 'title_lemma', 'description_lemma',
       'distribution_lemma', 'embedding_openai'],
      dtype='object')

In [93]:
df["embedding_openai"][0]

[-0.055915284901857376,
 0.011851861141622066,
 -0.010663777589797974,
 0.005630358587950468,
 -0.034332726150751114,
 0.04221465066075325,
 -0.03398499637842178,
 0.026334894821047783,
 0.03136541694402695,
 0.040452808141708374,
 0.011104238219559193,
 -0.02313576079905033,
 0.05308706685900688,
 -0.0077544208616018295,
 0.015358158387243748,
 0.023541448637843132,
 -0.006728611886501312,
 0.03171314671635628,
 0.027331724762916565,
 0.08679387718439102,
 0.05878986790776253,
 -0.029881758615374565,
 -0.03899233415722847,
 0.038575056940317154,
 0.0490533784031868,
 -0.037021853029727936,
 0.04117145389318466,
 -0.02120005339384079,
 0.05141795426607132,
 0.004007610026746988,
 0.08549568057060242,
 -0.023367581889033318,
 -0.026288529857993126,
 0.030438130721449852,
 -0.024781692773103714,
 0.02538442797958851,
 -0.028514014557003975,
 0.04648016020655632,
 -0.02589443325996399,
 0.00331794167868793,
 0.006641678977757692,
 -0.035329557955265045,
 -0.01047832053154707,
 0.035028189

In [94]:
df.head(2)

,identifier,link,title,description,keyword,distribution,is_study,text_for_embedding,title_lemma,description_lemma,distribution_lemma,embedding_openai
0,a6828125-ae95-404f-9fb0-52eb91302fd8,https://datenregister.berlin.de/api/3/action/c...,Suche nach anerkannten Veranstaltungen,In Berlin anerkannte Weiterbildungsveranstaltu...,bildungszeit simplesearch weiterbildung weiter...,"None Webseite von ""Suche nach anerkannten Vera...",False,Suche nach anerkannten Veranstaltungen In Berl...,Suche nach anerkannt Veranstaltung,in Berlin anerkannt Weiterbildungsveranstaltung .,None Webseite von Suche nach anerkannt Veranst...,"[-0.055915284901857376, 0.011851861141622066, ..."
1,96643363-971f-43f9-b93e-e721c5b84872,https://datenregister.berlin.de/api/3/action/c...,VBB-Fahrplandaten via GTFS,Der Verkehrsverbund Berlin-Brandenburg (VBB) s...,Bahn Bus Fahrplan Fähre S-Bahn Straßenbahn Tra...,VBB-Fahrplandaten,False,VBB-Fahrplandaten via GTFS Der Verkehrsverbund...,VBB Fahrplandaten via GTFS,der Verkehrsverbund Berlin Brandenburg VBB ste...,VBB Fahrplandaten,"[-0.046875447034835815, -0.03891636058688164, ..."


In [97]:
cols = [
    "identifier",
    "link",
    "title",
    "title_lemma",
    "description",
    "description_lemma",
    "text_for_embedding",
    "keyword",
    "distribution",
    "distribution_lemma",
    "is_study",
    "embedding_openai",
]

df = df[cols]

In [98]:
df.head(2)

,identifier,link,title,title_lemma,description,description_lemma,text_for_embedding,keyword,distribution,distribution_lemma,is_study,embedding_openai
0,a6828125-ae95-404f-9fb0-52eb91302fd8,https://datenregister.berlin.de/api/3/action/c...,Suche nach anerkannten Veranstaltungen,Suche nach anerkannt Veranstaltung,In Berlin anerkannte Weiterbildungsveranstaltu...,in Berlin anerkannt Weiterbildungsveranstaltung .,Suche nach anerkannten Veranstaltungen In Berl...,bildungszeit simplesearch weiterbildung weiter...,"None Webseite von ""Suche nach anerkannten Vera...",None Webseite von Suche nach anerkannt Veranst...,False,"[-0.055915284901857376, 0.011851861141622066, ..."
1,96643363-971f-43f9-b93e-e721c5b84872,https://datenregister.berlin.de/api/3/action/c...,VBB-Fahrplandaten via GTFS,VBB Fahrplandaten via GTFS,Der Verkehrsverbund Berlin-Brandenburg (VBB) s...,der Verkehrsverbund Berlin Brandenburg VBB ste...,VBB-Fahrplandaten via GTFS Der Verkehrsverbund...,Bahn Bus Fahrplan Fähre S-Bahn Straßenbahn Tra...,VBB-Fahrplandaten,VBB Fahrplandaten,False,"[-0.046875447034835815, -0.03891636058688164, ..."


In [99]:
df.sample(1).T

,442
identifier,f7e4c358-989b-4c0f-8d47-7ccbb921b459
link,https://datenregister.berlin.de/api/3/action/c...
title,ATKIS Leitung (Linien) - [WFS]
title_lemma,ATKIS Leitung Linie WFS
description,Eine aus Drähten oder Fasern hergestellte Leit...
description_lemma,ein aus Dräht oder Faser hergestellt Leitung z...
text_for_embedding,ATKIS Leitung (Linien) - [WFS] Eine aus Drähte...
keyword,ATKIS Basis-DLM Berlin DLM Digitales Landschaf...
distribution,Endpunkt-Beschreibung des WFS-Service Maschine...
distribution_lemma,Endpunkt Beschreibung der WFS Service Maschine...


In [100]:
df.to_parquet(DATA_WITH_EMBEDDINGS)
df.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
Index: 3311 entries, 0 to 3310
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   identifier          3311 non-null   object
 1   link                3311 non-null   object
 2   title               3311 non-null   object
 3   title_lemma         3311 non-null   object
 4   description         3311 non-null   object
 5   description_lemma   3311 non-null   object
 6   text_for_embedding  3311 non-null   object
 7   keyword             3311 non-null   object
 8   distribution        3311 non-null   object
 9   distribution_lemma  3311 non-null   object
 10  is_study            3311 non-null   bool  
 11  embedding_openai    3311 non-null   object
dtypes: bool(1), object(11)
memory usage: 52.2 MB


# Test embeddings


Perform a quick check with scikit's cosine similarity function.


In [4]:
df = pd.read_parquet(DATA_WITH_EMBEDDINGS)

In [7]:
query = "fahrräder in berlin"
show_n_results = 20

# Vector search based on OpenAI embeddings.
# now = time()
embedding_openai = embed_with_openai(query)
cosine_sim = cosine_similarity(embedding_openai, df.embedding_openai.tolist())
top_k = np.argsort(cosine_sim[0])
top_k = top_k[::-1]
display(df.iloc[top_k][["title"]].head(show_n_results).values)

RuntimeError: Mismatch in number of embeddings (1) and input texts (19)

In [8]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from typing import List


def search_similar_documents(query: str, df, n_results: int = 20) -> List[str]:
    """
    Search for documents similar to the query using cosine similarity.

    Args:
    query (str): The search query.
    df (pd.DataFrame): DataFrame containing document embeddings.
    n_results (int): Number of results to return.

    Returns:
    List[str]: Titles of the most similar documents.
    """
    # Generate embedding for the query
    query_embedding = np.array(embed_with_openai([query])[0]).reshape(1, -1)

    # Ensure df.embedding_openai contains lists, not strings
    if isinstance(df.embedding_openai.iloc[0], str):
        df["embedding_openai"] = df["embedding_openai"].apply(eval)

    # Convert DataFrame embeddings to 2D numpy array
    doc_embeddings = np.array(df.embedding_openai.tolist())

    # Compute cosine similarity
    cosine_sim = cosine_similarity(query_embedding, doc_embeddings)[0]

    # Get indices of top results
    top_indices = np.argsort(cosine_sim)[::-1][:n_results]

    # Return titles of top results
    return df.iloc[top_indices]["title"].tolist()


# Usage
query = "fahrräder in berlin"
show_n_results = 20

similar_documents = search_similar_documents(query, df, show_n_results)

# Display results
for i, title in enumerate(similar_documents, 1):
    print(f"{i}. {title}")

1. 14 Radrouten und Radverkehrsanlagen - GPS-Tracks für die Radrouten durch Berlin
2. Fahrradstraßen - [WFS]
3. Fahrradstraßen - [WMS]
4. Radverkehrsmaßnahmen in Berlin
5. Radzähldaten in Berlin
6. Fahrraddiebstahl in Berlin
7. Fahrradreparaturstationen - [WFS]
8. Radverkehrsnetz - [WFS]
9. Berlin zählt Mobilität
10. Reparaturführer Charlottenburg-Wilmersdorf
11. Berliner und Brandenburger Wochen- und Trödelmärkte
12. Fahrradreparaturstationen - [WMS]
13. E-Mobility Ladesäulen in Berlin
14. Radverkehrsnetz - [WMS]
15. Berliner und Brandenburger Straßen- und Volksfeste
16. Standorte der Verkehrsinformationstafeln in Berlin
17. Berliner und Brandenburger Weihnachtsmärkte
18. Parkverbotszonen für Mietfahrzeuge der Mikromobilität - [WFS]
19. Berliner Straßen- und Volksfeste 2017
20. Parkverbotszonen für Mietfahrzeuge der Mikromobilität - [WMS]
